In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print(f"Model loaded successfully. Max sequence length: {max_seq_length}")

In [ ]:
from datasets import load_dataset

full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")
print(f"Total training samples: {len(full_dataset)}")

shuffled_dataset = full_dataset.shuffle(seed=42)
train_size = int(len(shuffled_dataset) * 0.9)
train_dataset = shuffled_dataset.select(range(train_size))
validation_dataset = shuffled_dataset.select(range(train_size, len(shuffled_dataset)))

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(validation_dataset)}")

In [ ]:
training_prompt = """Analyze the math problem and solution. Output 'True' if correct, 'False' if incorrect.

Question: {}
Solution: {}
Answer: {}
Output: {}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt.format(
            question,
            str(solution),
            str(answer),
            str(output)
        ) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

print("\nFormatting datasets...")
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_val_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

print("Sample formatted text:")
print(formatted_train_dataset[0]["text"][:300])
print(f"\nDataset columns: {formatted_train_dataset.column_names}")

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params:,} ({100 * trainable_params / all_params:.2f}%)")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        eval_strategy = "steps",
        eval_steps = 200,
        save_strategy = "steps",
        save_steps = 200,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
        load_best_model_at_end = True,
        metric_for_best_model = "loss",
    ),
)

print(f"Total steps: ~{len(formatted_train_dataset) * 2 // (2 * 4)}")

In [ ]:
trainer.train()

In [ ]:

FastLanguageModel.for_inference(model)

inference_prompt = """Analyze the math problem and solution. Output 'True' if correct, 'False' if incorrect.

Question: {}
Solution: {}
Answer: {}
Output: """

def parse_output(response_text):
    try:
        output_part = response_text.split("Output:")[-1].strip().lower()
        if "true" in output_part[:20]:
            return True
        elif "false" in output_part[:20]:
            return False
        return "true" in output_part
    except:
        return False

correct = 0
total = min(100, len(validation_dataset))

print(f"\nEvaluating on {total} validation samples...")
for i in range(total):
    example = validation_dataset[i]
    question = example["question"]
    solution = example["solution"]
    answer = example["answer"]

    prompt = inference_prompt.format(question, str(solution), str(answer))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10, use_cache=True, temperature=0.1)
    response_text = tokenizer.batch_decode(outputs)[0]

    prediction = parse_output(response_text)
    actual = example["is_correct"]

    if prediction == actual:
        correct += 1

    if i < 5:
        print(f"\n--- Sample {i+1} ---")
        print(f"Predicted: {prediction}, Actual: {actual}, Match: {prediction == actual}")

accuracy = correct / total
print(f"\n{'='*60}")
print(f"Validation Accuracy: {accuracy:.4f} ({correct}/{total})")
print(f"{'='*60}")

if accuracy < 0.55:
    print("WARNING: Accuracy below 0.55 - consider retraining with different hyperparameters")
elif accuracy < 0.726:
    print("INFO: Accuracy below baseline (0.726) - room for improvement")
elif accuracy >= 0.80:
    print("EXCELLENT: Accuracy above 0.80!")

In [ ]:

try:
    from google.colab import drive
    drive.mount('/content/drive')
    save_path = "/content/drive/MyDrive/llama3_math_verifier"
except:
    save_path = "./saved_model"

import os
os.makedirs(save_path, exist_ok=True)

print(f"Saving model to {save_path}...")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:

import pandas as pd
from tqdm import tqdm

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
print(f"Test samples: {len(test_dataset)}")

predictions = []
true_count = 0
false_count = 0

for example in tqdm(test_dataset, desc="Generating predictions"):
    question = example["question"]
    solution = example["solution"]
    answer = example["answer"]

    prompt = inference_prompt.format(question, str(solution), str(answer))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10, use_cache=True, temperature=0.1)
    response_text = tokenizer.batch_decode(outputs)[0]

    prediction = parse_output(response_text)
    predictions.append(prediction)

    if prediction:
        true_count += 1
    else:
        false_count += 1

submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

submission.to_csv('submission.csv', index=False)

print(f"Total predictions: {len(predictions)}")
print(f"True predictions: {true_count} ({100*true_count/len(predictions):.2f}%)")
print(f"False predictions: {false_count} ({100*false_count/len(predictions):.2f}%)")
print(f"\nSubmission file 'submission.csv' created!")
print(f"{'='*60}")

print("\nFirst 10 predictions:")
print(submission.head(10))